# HPE Container Platform API series - Part 1
## Getting starting with the HPE Container Platform REST API


**Requirements:**
- HPE Container Platform deployment
- IP address of FQDN of the HPE Container Platform's controller host
- a tenant user and tenant admin account   

**Utilities:**   
- Postman, cURL  
- Jupyter Notebook server with bash kernel installed

**Definitions:**
- *HPE Container Platform* is an enterprise-grade container platform designed to deploy both cloud-native and non-cloud-native applications whether on-premises, at the edge, in multiple public clouds, or in a hybrid model. This makes the HPE Container Platform ideal for helping application developers and data scientists accelerate their application development and deployment on **containers**, on-demand through a self-service portal and a RESTful API that surfaces programmable access. To learn more about HPE Container Platform visit the [HPE DEV portal](https://developer.hpe.com) and check out the blog articles.

- *tenant:* A tenant is a group of users created by the platform administrator. A tenant can represent for example, an office location, a business unit, an organization, or a project. A tenant is allocated a set of resources (CPU/GPU, RAM, Storage, App Store images, or Kubernetes clusters) by the platform administrator. All the resources used by a tenant are not shared with other tenants. A tenant user is granted the role of member or admin for the tenant.


## The HPE Container Platform API Reference
The HPE Container Platform REST API allows you to achieve multiple actions programmatically, from performing administrative tasks, deploying cloud-native and non-cloud-native applications to scoring trained Machine Learning models.

Before you can call the HPE Container Platform API, you need to know what calls can be placed. The REST API reference documentation describes each object type, along with the supported operations, request input parameters, response model and response codes. 
To access the REST API reference documentation, obtain the IP address or hostname of the current active HPE Container Platform controller host from the administrator of the platform. Then in a web browser, navigate to the following URL:

``` 
http(s)://[Controller-IP-address-name]/apidocs
```
Accessing the HPE Container Platform API reference documentation varies depending on whether your platform has been configured to use HTTPS secure protocol or non-secure HTTP protocol.

All the REST API calls are in the form: 
``` 
An HTTP VERB such as (GET , POST, DELETE, PUT, PATCH, UPDATE),  
A target API object: http(s)://[Controller-IP-address]:8080/api/v2/[object]
```

## Session Authentication
With the exception of some API calls, most of the REST API calls you can do against the HPE Container Platform API requires authentication. HPE Container Platform uses a *‘session location’* to use as operation context. In a multi-tenant environment, you request an authentication session location by issuing an authentication request in the following form:
* Call the API to request (POST) a new login session, providing username/password credentials as well as the Tenant name in the JSON body. The user must be a valid tenant user credentials with a role (member or admin) in the requested tenant. 
* Extract the resource path of the created *session location* object from the JSON response header,
* For each subsequent call, set a new HTTP Header with its key set to *X-BDS-SESSION* and its value set to the session location value and used as the *working tenant* context.  

``` 
Note: the session location will expire after 24 hours. 
```   

If you are not already familiar with REST API calls and Postman, I encourage you to check-out the [Understanding API basics and the value they provide](https://developer.hpe.com/blog/understanding-api-basics-and-the-value-they-provide) blog on HPE Developer Community portal. It explains you REST API concepts such as HTTP verbs you call against the API, the headers, and payloads and how to use Postman to make API calls.  

### Using Postman

Postman can be used to issue REST API calls to the HPE Container Platform. The following capture shows the information passed by Postman to the API. It is a POST operation against **/api/v2/session** to create a session object. The header specifies JSON as the exchange format and the body provides the user credentials and tenant name. The target HPE Container Platform controller IP address, the user credentials and tenant name are defined in a group of variables called a Postman *Environment*. These variables typically describe a given target environment and are expected to be set before the API calls. Postman also allows you to extract the session location from the response header and store it in a Postman environment variable for later use. This is accomplished by using a simple JavaScript code as shown in the *Tests* tab. 

<img src="Pictures/HPECP-Postman-post-session.gif" height="800" width="600" align="left">

Save the value of the session _Location_ from the response header above, to use it in all your subsequent API calls. For example, here we call the HPE Container Platform API to get the list of application templates available to our tenant membership.

<img src="Pictures/HPECP-Postman-Get-template.gif" height="800" width="600" align="left">

Let's do the equivalent API call to create the session using cURL. Information on cURL can be found [here](https://curl.haxx.se/)

### Using cURL

#### Initialize the environment.
   
**IMPORTANT: Before running the next code cells, please make sure to adjust the environment variables below according to your tenant username and password**

In [1]:
#
# environment variables to be adjusted by the student
#
##username="studentXX" # your tenant login credentials - username and password (it matches your Notebook Student account)
##password="thepassword"
username="demouser"
password="password"
#
# fixed environment variables setup by the HPE CP administrator
#
controller_endpoint="controller.hpedevlab.net:8080"
tenantname="HackShack Tenant"

#### -1- Authenticate as Tenant user in the specified tenant:

In [2]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo $SessionId

This is your session location:  /api/v2/session/cfd5f7a8-d23e-414e-8daa-738a37a20d72
cfd5f7a8-d23e-414e-8daa-738a37a20d72


#### -2- Making subsequent API calls:
For each subsequent call, you must set a new HTTP Header with its key set to **X-BDS-SESSION** and its value set to the session location value. This session token is used as the *working tenant* context.

In [3]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/template" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

{
  "_links": {
    "self": {
      "href": "/api/v2/template"
    }
  },
  "_embedded": {
    "templates": [
      {
        "_links": {
          "self": {
            "href": "/api/v2/template/2"
          },
          "all_templates": {
            "href": "/api/v2/template"
          }
        },
        "_embedded": {
          "label": {
            "_links": {
              "self": {
                "href": "/api/v2/template/2/label"
              }
            },
            "name": "Centos 7 example",
            "description": "system-created example template"
          },
          "clusterspec": {
            "_links": {
              "self": {
                "href": "/api/v2/template/2/clusterspec"
              }
            },
            "nodegroup": {
              "catalog_entry_distro_id": "bluedata/centos7",
              "role_configs": [
                {
                  "role_id": "controller",
                  "flavor_name": "Large"
                },
     

### -3- Closing your login session:

Although session have a time to live (TTL) of 24 hours, it is **best practice** in REST API programming to cleanup and delete those sessions when done. You can use a DELETE /api/v2/session/SessionId to achieve this.

In [4]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Mon, 06 Apr 2020 18:59:24 GMT
Server: BlueData EPIC 5.0



The status *204 No Content* means the session has been deleted.

## Summary

In this tutorial, you learned how to authenticate to the HPE Container Platform API endpoint and retrieve object data by providing the authentication session location in the header.